# Prepare Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [11]:
datapath = r'/content/drive/Othercomputers/Spectre2/DBA/Data/'

In [12]:
df_qafinal = pd.read_csv(datapath + 'QA_Final.csv')
print(df_qafinal.shape)
df_qafinal.head(3)

(1119, 6)


,QuesId,SectionId,ContextId,Question,Answer,Reference
0,1,Book01_001,1,Question: Who composed the Mahabharata and rec...,Answer: The Mahabharata was composed by Krishn...,"Reference: ""Sauti said, 'Having heard the dive..."
1,2,Book01_001,1,Question: What is the nature of the Mahabharat...,"Answer: The Mahabharata is a sacred work, an e...","Reference: ""The Purana, first promulgated by t..."
2,3,Book01_001,1,Question: How did creation begin according to ...,"Answer: At the beginning of the Yuga, from a m...","Reference: ""In this world, when it was destitu..."


In [13]:
df_prompt_qa = pd.read_csv(datapath + "Question_Prompts_with_QA.csv")
print(df_prompt_qa.shape)
df_prompt_qa.head(3)

(7315, 7)


,Context_Len,Len,Section_Id,Context_Id,Prompt,QA,QA2
0,6000,5985,Book01_001,1,"Write 3 questions, corresponding answers, and ...",Question: Who composed the Mahabharata and rec...,NaN
1,6000,5674,Book01_001,2,"Write 3 questions, corresponding answers, and ...",Question: How will the universe be affected at...,NaN
2,6000,5169,Book01_001,3,"Write 3 questions, corresponding answers, and ...",Question: Who became the writer of Vyasa's wor...,NaN


In [3]:
df_prompt_qa  = df_prompt_qa.loc[df_prompt_qa.QA.notna()][['Section_Id','Context_Id','Prompt']]
df_prompt_qa['Context'] = df_prompt_qa['Prompt'].str.replace('Text: ', "|").str.replace('Question: \n', "|").str.split("|").str[1].str.strip()

df_prompt_qa = df_prompt_qa[['Section_Id','Context_Id', 'Context']]

df_qafinal = df_qafinal.merge(df_prompt_qa, left_on="ContextId", right_on="Context_Id", how="left")
df_qafinal['Question'] = df_qafinal.Question.str.replace('Question: ','')
df_qafinal['Answer'] = df_qafinal.Answer.str.replace('Answer: ','')
df = df_qafinal[['QuesId','Question','Answer','Context']]


In [ ]:
df.rename(columns={"Question" : "question","Answer" : "answer", "Context" : "context"},inplace=True)
df['ques_len'] = df.question.str.len()
df['ans_len']  = df.answer.str.len()
df['contex_len']  = df.context.str.len()

df.to_csv('/content/drive/MyDrive/Booksummary/qa_merged.csv')


# Create Embedding

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')



In [ ]:
#sentences = ["This is an example sentence", "Each sentence is converted"]

ques_sentences = df.question.tolist()
ques_embeddings = model.encode(ques_sentences)

ans_sentences = df.answer.tolist()
ans_embeddings = model.encode(ans_sentences)

context_sentences = df.context.tolist()
context_embeddings = model.encode(context_sentences)



In [ ]:
qe= torch.tensor(ques_embeddings, dtype=torch.float).to(DEVICE)
ae= torch.tensor(ans_embeddings, dtype=torch.float).to(DEVICE)
ce= torch.tensor(ques_embeddings, dtype=torch.float).to(DEVICE)

In [ ]:
# Flatten the tensors into 1D arrays
qe_list = qe.tolist()
ae_list = ae.tolist()
ce_list = ce.tolist()

In [ ]:
df_embed = pd.DataFrame({"quest_embed": qe_list, "ans_embed": ae_list, "context_embed" : ce_list})
df_embed.to_csv("/content/drive/MyDrive/Booksummary/embedding.csv")

In [ ]:
df_embed